# Studio Esecutivo
#### Filippo Iacobelli e Luca Rossicone

Dallo studio preliminare è emerso come gli ampi tempi di esecuzione siano dovuti alle funzioni esterne `cuboidGrid` e `simplexGrid`.
Abbiamo dunque tentato di aggirare queste funzioni andando a leggere i risultati direttamente da file. Offline vengono creati dei file di testo (in realtà si tratta di Artifacts) contenenti i risulati di `cuboidGrid` e `simplexGrid` con input standard e durante l'esecuzione le funzioni del modulo mapper dovranno solamente leggere i risultati da file. 
Visto che eseguiamo operazioni di I/O abbiamo pensato che sarebbe stato opportuno dividere le operazioni in Tasks o Threads per ottenere miglioramenti nelle prestazioni.

In [ ]:
using DelimitedFiles
using Pkg.Artifacts
using DataStructures
using LinearAlgebraicRepresentation
using IntelVectorMath
using BenchmarkTools
Lar = LinearAlgebraicRepresentation
IVM = IntelVectorMath

La funzione `simplifyCellsOpt` resta identica a quella realizzata nello studio precedente.

In [ ]:
function simplifyCellsOpt(V,CV)::Tuple{Matrix{Float64},Vector{Vector{Int64}}}
	PRECISION = 5
	vertDict = DefaultDict{Array{Float64,1}, Int64}(0)
	index = 0
	W = Array{Float64,1}[]
	FW = Array{Int64,1}[]
	@inbounds @simd for incell in CV
		outcell = Int64[]
		@inbounds @simd for v in incell
			vert = @view V[:,v]
			key = map(Lar.approxVal(PRECISION), vert)
			if vertDict[key]==0
				index += 1
				vertDict[key] = index
				push!(outcell, index)
				push!(W,key)
			else
				push!(outcell, vertDict[key])
			end
		end
		append!(FW, [[Set(outcell)...]])
	end
	return hcat(W...),FW
end

Come detto in precedenza scriviamo i domini necessari alle esecuzioni delle successive funzioni su file in formato txt. Poiché la funzione `writedlm` è adatta a scrivere matrici, nel caso degli spigoli siamo stati costretti a modificarne leggeremente il formato per poi ripristrinarlo successivamente.

In [ ]:
function writeDomain(shapeCircle=[36],shapeToroidal=[24,36],shapeCuboid=[1,1,1])
    V,EV = Lar.cuboidGrid(shapeCircle)
    if !isdir("../domain")
        mkdir("../domain")
    end
    cd("..")
    open("domain/circleVertex.txt", "w") do io
        writedlm(io, V)
    end
    open("domain/circleEdge.txt", "w") do io
        writedlm(io, hcat(EV...))
    end
    V,CV = Lar.simplexGrid(shapeToroidal)
    open("domain/toroidalVertex.txt", "w") do io
        writedlm(io, V)
    end
    open("domain/toroidalEdge.txt", "w") do io
        writedlm(io, hcat(CV...))
    end
    V,EV = Lar.cuboidGrid(shapeCuboid)
    open("domain/cuboidVertex.txt", "w") do io
        writedlm(io, V)
    end
    open("domain/cuboidEdge.txt", "w") do io
        writedlm(io, hcat(EV...))
    end
end

Realizziamo a partire dai file di testo degli Artifacts (per maggiori informazioni si può leggere la relazione). Viene creato anche il file
`Artifacts.toml` contenente l'hash dell'artifact e il link dove è possibile reperirlo qualora non fosse disponibile localmente.

In [ ]:
function createArtifacts()
    cd("..")
    hash = create_artifact() do dir
        cp("domain/circleVertex.txt", joinpath(dir, "circleVertex.txt"))
        cp("domain/circleEdge.txt", joinpath(dir, "circleEdge.txt"))
        cp("domain/toroidalVertex.txt", joinpath(dir, "toroidalVertex.txt"))
        cp("domain/toroidalEdge.txt", joinpath(dir, "toroidalEdge.txt"))
        cp("domain/cuboidVertex.txt", joinpath(dir, "cuboidVertex.txt"))
        cp("domain/cuboidEdge.txt", joinpath(dir, "cuboidEdge.txt"))
    end
    tarball_hash = archive_artifact(hash, "domain.tar.gz")
    bind_artifact!("Artifacts.toml", "domain", hash,
        download_info=[("https://bitbucket.org/zoso9999/linearalgebraicrepresentation/src/filippo/domain.tar.gz", 
                        tarball_hash)])
end 


In [ ]:
# writeDomain()
# createArtifacts()

Con le funzioni `readArtifacts*` recuperiamo i domini salvati sugli artifacts e ne ripristiniamo l'opportuno formato (matrice e vettore di vettori).

In [ ]:
function readArtifactsCircle()
    vertex = joinpath(artifact"domain", "circleVertex.txt")
    V = readdlm(vertex, '\t', Float64, '\n', use_mmap=true, dims=(1,37))
    edge = joinpath(artifact"domain", "circleEdge.txt")
    ev = readdlm(edge, '\t', Int64, '\n', use_mmap=true, dims=(2,36))
    EV = Array{Int64,1}[]
    for col in eachcol(ev)
        push!(EV,col)
    end
    V,EV
end


Confrontando i tempi fra l'eecuzione della funzione `simplexGrid` e `readArtifacts*` è possibile notare come almeno ad una prima esecuzione lo speedup ottenuto è notevole.

In [ ]:
@time readArtifactsCircle()
@time Lar.simplexGrid([36])

Già ad una seconda esecuzione la differenza non è più così ampia. In seguito ad una prima esecuzione infatti i risultati vengono salvati in cache e una successiva chiamata evita che la funzione venga rieseguita per intero ottimizzando di molto i tempi. Per questa ragione abbiamo preferito utilizzare la macro `@time` che prende i tempi su una singola esecuzione piuttosto che `@btime` che invece lancia la funzione più volte.

In [ ]:
@time readArtifactsCircle()
@time Lar.simplexGrid([36])

Per migliorare ancor di più le prestazioni abbiamo pensato di realizzare più Tasks o Threads e misuare i tempi dopo queste ottimizzazioni.

In [ ]:
function readArtifactsToroidal()
    ev = readdlm(joinpath(artifact"domain", "toroidalEdge.txt"), '\t', Int64, '\n', use_mmap=true, dims=(3,1728))
    V = readdlm(joinpath(artifact"domain", "toroidalVertex.txt"), '\t', Float64, '\n', use_mmap=true, dims=(2,925))
    EV = Array{Int64,1}[]
    for col in eachcol(ev)
        push!(EV,col)
    end
    V,EV
end

In [ ]:

function readArtifactsToroidalThreads() 
    edge() = readdlm(joinpath(artifact"domain", "toroidalEdge.txt"), '\t', Int64, '\n', use_mmap=true, dims=(3,1728))
    vertex() = readdlm(joinpath(artifact"domain", "toroidalVertex.txt"), '\t', Float64, '\n', use_mmap=true, dims=(2,925))
    t1 = Base.Threads.@spawn edge()
    t2 = Base.Threads.@spawn vertex()
    EV = Array{Int64,1}[]
    ev = fetch(t1)
    for col in eachcol(ev)
        push!(EV,col)
    end
    V = fetch(t2)
    V,EV
end

In [ ]:
function readArtifactsToroidalTask() 
    edge() = readdlm(joinpath(artifact"domain", "toroidalEdge.txt"), '\t', Int64, '\n', use_mmap=true, dims=(3,1728))
    vertex() = readdlm(joinpath(artifact"domain", "toroidalVertex.txt"), '\t', Float64, '\n', use_mmap=true, dims=(2,925))
    @async edge()
    @async vertex()
    EV = Array{Int64,1}[]
    ev = edge()
    @sync for col in eachcol(ev)
        push!(EV,col)
    end
    V = vertex()
    V,EV
end

Grazie alle operazioni di I/O che possono essere facilmente parellelizzate, si può notare come la soluzione multiple threads sia la più efficace seguita da quella con più task. In questo caso la macro `@btime` non crea problemi poiché non vi sono risulati di operazioni che vengono salvati e dunque è possibile utilizzarla per effettuare benchmark.

In [ ]:
@btime readArtifactsToroidal()
@btime readArtifactsToroidalThreads()
@btime readArtifactsToroidalTask()

Di seguito cerchiamo di analizzare lo speedup ottenuto dalle funzioni prese come modelli nello studio preliminare utilizzando però l'implmentazione appena descritta per il calcolo dei domini. Si nota come questo oscilli fra il *4x* e l'*8x* a seconda dei casi.  

In [ ]:
function circle(radius=1., angle=2*pi)
    function circle0(shape=[36])
        V, EV = Lar.cuboidGrid(shape)
        V = (angle/shape[1])*V
        V = hcat(map(u->[radius*cos(u); radius*sin(u)], V)...)
        W, EW = Lar.simplifyCells(V, EV)
        return W, EW
    end
    return circle0
end

In [ ]:
function circleOpt(radius=1., angle=2*pi)
    V,EV = readArtifactsCircle()
    V = (angle/size(EV)[1])*V
    V = vcat(radius*IVM.cos(V),radius*IVM.sin!(V))
    W, EW = simplifyCellsOpt(V, EV)
    return W, EW
end

In [ ]:
@time circleOpt()
@time circle()()

In [ ]:

function toroidal(r=1., R=2., angle1=2*pi, angle2=2*pi)
    function toroidal0(shape=[24,36])
        V, CV = Lar.simplexGrid(shape)
        V = [angle1/(shape[1]) 0;0 angle2/(shape[2])]*V
        W = [V[:, k] for k=1:size(V, 2)]
        V = hcat(map(p->let(u, v)=p;[(R+r*cos(u))*cos(v);
          (R+r*cos(u))*sin(v);-r*sin(u)]end, W)...)
        W, CW = Lar.simplifyCells(V, CV)
        return W, CW
    end
    return toroidal0
  end

In [ ]:
function toroidalOpt(r=1., R=2., angle1=2*pi, angle2=2*pi)
    shape=[24,36]
    V, CV = readArtifactsToroidalThreads()
    V = [angle1/(shape[1]) 0;0 angle2/(shape[2])]*V
    U = V[1,:]; Z = V[2,:]
    sinU = IVM.sin(U); sinZ = IVM.sin(Z)
    IVM.cos!(U); IVM.cos!(Z)
    tmp = r*U.+R
    V = hcat(tmp.*Z, tmp.*sinZ, -r*sinU)
    W, CW = simplifyCellsOpt(V', CV)
    W, CW
end

In [18]:
@time toroidal()()
@time toroidalOpt()

In [ ]:
function readArtifactsCuboid()
    vertex = joinpath(artifact"domain", "cuboidVertex.txt")
    V = readdlm(vertex, '\t', Float64, '\n', use_mmap=true, dims=(3,8))
    edge = joinpath(artifact"domain", "cuboidEdge.txt")
    ev = readdlm(edge, '\t', Int64, '\n', use_mmap=true, dims=(8,1))
    EV = Array{Int64,1}[]
    for col in eachcol(ev)
        push!(EV,col)
    end
    V,EV
end

In [ ]:
function cuboid(maxpoint=[1,1,1], full=false,
    minpoint::Array=zeros(length(maxpoint)))
    @assert( length(minpoint) == length(maxpoint) )
    dim = length(minpoint)
    shape = ones(Int, dim)
    cell = Lar.cuboidGrid(shape, full)
    size = maxpoint - minpoint
    out = Lar.apply(Lar.t(minpoint...) * Lar.s(size...), cell)
end

In [ ]:
function cuboidOpt(maxpoint=[1,1,1], full=false,
    minpoint::Array=zeros(length(maxpoint)))::
    Tuple{Matrix{Float64},Vector{Vector{Int64}}}
    @assert length(minpoint) == length(maxpoint)
    dim = length(maxpoint)
    shape = ones(Int, dim)
    cell = readArtifactsCuboid()
    out = Lar.apply(Lar.t(minpoint...) * Lar.s(maxpoint...), cell)
end

In [ ]:
@time cuboid()
@time cuboidOpt()

[*Link al repository del progetto*](https://github.com/Asprofumo/mapper.jl/)